In [1]:
import numpy as np
import pandas as pd
from data_util import *
from tqdm.notebook import tqdm
from neighbor_tag import Neighbor

/Users/jehunyoo/AI/kakao/MelonPlaylistContinuation/main/neighbor_tag.py:6: UserWarning: Unsupported module 'tqdm' is used.
  warn("Unsupported module 'tqdm' is used.")


In [2]:
### 1. load data
train = pd.read_json("res/train.json")
val = pd.read_json("res/val.json")
# test = pd.read_json("res/test.json")

In [3]:
### 2. data preprocessing
tag_to_id, id_to_tag = tag_id_meta(train, val)
train = convert_tag_to_id(train, tag_to_id)
val = convert_tag_to_id(val  , tag_to_id)
display(train, val)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[0],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[1, 2]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[3, 4]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[15],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[2083, 2287, 36, 1023, 34, 0, 22994]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[25],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[9387, 455, 474, 447, 5836, 9388, 30, 1223]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[4, 210, 8561, 18, 5685, 187]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,[4],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[11913, 335, 3162, 455, 23086]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [4]:
### 3. modeling
### 3.1 hyperparameters: pow_alpha, pow_beta
pow_alpha = 1.0
pow_beta = 0.0

In [5]:
### 4. range setting - Neighbor.predict()
### 4.1 range(start, end); if end == None, then range(start, end of val)
### 4.2 auto_save: boolean; False(default)
### 4.3 return type of Neighbor.predict() : pandas.DataFrame
pred = Neighbor(pow_alpha=pow_alpha, pow_beta=pow_beta, \
                train=train, val=val).predict(start=0, end=None, auto_save=False)
pred = convert_id_to_tag(pred, id_to_tag)

version = Neighbor.__version__
version = version[version.find('-') + 1: version.find('.')]
path = f"submission/neighbor-tag/neighbor-tag_beta1_a{int(pow_alpha * 10)}b{int(pow_beta * 10)}"
fname = f"neighbor{version}_a{int(pow_alpha * 10)}b{int(pow_beta * 10)}"
pred.to_json(f'{path}/{fname}.json', orient='records')

display(pred)

Neighbor version: Neighbor-tag_beta1.0


100%|██████████| 23015/23015 [20:12<00:00, 18.98it/s]


,id,songs,tags
0,118598,[],[]
1,131447,[],[]
2,51464,[],[]
3,45144,[],[]
4,79929,[],[]
...,...,...,...
23010,101722,[],"[휴식, 새벽, 밤, 힐링, 기분전환, 감성, 까페, 비오는날, 추억, 회상]"
23011,122127,[],"[엄마, 아버지, 부모님, 위로가_되는_음악, 너무힘들어, 사랑, 어버이날, 위로,..."
23012,77438,[],[]
23013,36231,[],[]
